# Convert .hips image files to ENVI with PlantCV
**NOTE** Only compatible on Windows OS (https://pypi.org/project/videometer/)

We suggest using miniforge, which should already be installed on your Windows machine, and use this command line to setup a virtual environment to contain the software needed for image conversion. 

#### Environment setup

Deactivate any conda environments you have back to the base environment
Next, install the uv package manager:

```
pip install uv
```

Make a directory for a new virtual environment and then create one using uv:

```
mkdir videometer
cd videometer
uv venv --python 3.11
```

Then activate the new environment:

```
.venv\Scripts\activate
```

Install packages:

```
uv pip install plantcv videometer jupyterlab ipympl
```

And then launch a jupyter notebook:

```
jupyter lab
```

# Start here

In [1]:
### Update this path ### 
img_dir = "./imgs"

## Loop over images in the directory (set above) and convert to ENVI files, then save to subdirectory called "envi" 

In [5]:
# Imports
from plantcv import plantcv as pcv
import videometer.hips as hips
from pathlib import Path
import numpy as np
import os

# Initialize list of hips image files 
image_files = []

# Reset the current working directory
os.chdir(".")

# Loop through and add paths if they have a ".hips" file extension
for file in Path(img_dir).iterdir():
    if file.is_file() and file.suffix.lower() ==".hips":
        img_path = str(file)

        # Read in .hips file and stores as an ImageClass object
        plate = hips.read(path=img_path)
        
        # Read in .hips file and returns 3D NumPy array
        plate_array = hips.readOnlyPixelValues(path=img_path)

        ## Display the RGB image 
        #hips.showRGB(plate, False)
        
        # Store list of wavelengths 
        wl_1darray = plate.WaveLengths
        
        # Create dictionary of wavelengths
        wavelength_dict = {}
        for j, wavelength in enumerate(wl_1darray):
            wavelength_dict.update({float(wavelength): float(j)})

        # Create a plantcv spectral data object
        hips_spectral = pcv.Spectral_data(array_data = plate_array,
                                          max_wavelength=max(wl_1darray),
                                          min_wavelength=min(wl_1darray),
                                          max_value=np.max(plate_array),
                                          min_value=np.min(plate_array),
                                          d_type=plate_array.dtype,
                                          wavelength_dict=wavelength_dict,
                                          # x-dimension
                                          samples=plate.Width,
                                          # y-dimension
                                          lines=plate.Height,
                                          interleave=None,
                                          wavelength_units="nm",
                                          array_type="datacube",
                                          # Could store the RGB image but can also reconstruct with available bands
                                          pseudo_rgb=None,
                                          filename=img_path.split(".")[1],
                                          default_bands=None,
                                          # Store the size scaling information to metadata
                                          metadata={"MmPixel":plate.MmPixel})
        
        # Create an "envi" output folder if one doesn't already exist
        if not os.path.exists(os.path.join(img_dir, "envi")):
          os.makedirs(os.path.join(img_dir, "envi"))
            
        # Format the output filename based on the input image filename
        outfilepath = os.path.join(img_dir, "envi", os.path.basename(img_path))
        print(outfilepath)

        # Save out to file
        pcv.hyperspectral.write_data(outfilepath, hips_spectral)


./imgs\envi\Plate2-Copy1.hips
./imgs\envi\Plate2.hips
